<img src="asset/softmax.png" alt="softmax" width="350">


Softmax 回归（Softmax Regression），也称为多项逻辑回归（Multinomial Logistic Regression），是逻辑回归的一种扩展，用于解决多分类问题，即当输出类别数大于二时的分类任务。



# Softmax 与交叉熵损失详细解释

从图片内容来看，主要讲解了 **Softmax** 函数与 **交叉熵损失**（Cross-Entropy Loss）之间的关系，以及它们在分类任务中的作用和推导。下面将分点进行详细说明。

---

## 1. 交叉熵的概念

1. **定义**
   交叉熵（Cross Entropy）最初是信息论中的概念，用于衡量两个概率分布 \\(p\\) 和 \\(q\\) 之间的差异，定义为：

   $$
   H(p, q) = - \sum_i p_i \log (q_i).
   $$

   - \\(p\\) 通常表示「真实」分布
   - \\(q\\) 通常表示「模型预测」分布
   - \\(p_i\\) 和 \\(q_i\\) 分别表示在第 \\(i\\) 个类别上的真实概率和预测概率

2. **在分类中的含义**
   在机器学习的多分类任务中，真实标签往往使用 **one-hot** 编码。比如，对于 3 类分类问题，如果真实标签是第 2 类，那么真实标签向量 \\(\mathbf{y} = (0, 1, 0)\\)。此时：
   - \\(p\\) 就是 one-hot 分布
   - \\(q\\) 则是模型预测的概率分布

3. **为什么选交叉熵做损失**
   - 交叉熵能够直接衡量两个分布间的差异，当两个分布越接近时，交叉熵越小
   - 对于分类任务，「真实分布」是 one-hot，最小化交叉熵等价于最大化正确类别的预测概率

---

## 2. 将交叉熵用于损失函数

在分类任务中，通常将交叉熵作为损失函数来训练模型。若将 \\(\mathbf{y} = (y_1, y_2, \ldots, y_K)\\) 作为 one-hot 的真实标签，\\(\hat{\mathbf{y}} = (\hat{y}_1, \hat{y}_2, \ldots, \hat{y}_K)\\) 作为预测概率分布，则损失函数可写为：

$$
l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{i=1}^K y_i \log(\hat{y}_i).
$$

- 由于 \\(\mathbf{y}\\) 是 one-hot 编码，只有真实类别 \\(j\\) 的 \\(y_j = 1\\) 起作用，其余均为 0，因此损失可以简化为：
  $$
  l(\mathbf{y}, \hat{\mathbf{y}}) = -\log (\hat{y}_j).
  $$
- 当预测概率 \\(\hat{y}_j\\) 越大时，损失越小；当预测概率很小时，损失会很大。

---

## 3. Softmax 函数的作用

1. **定义**
   Softmax 函数用于将任意实数向量映射到 \\([0,1]\\) 的区间，且各输出之和为 1，从而得到一个概率分布。给定模型输出的向量 \\(\mathbf{o} = (o_1, o_2, \ldots, o_K)\\)，Softmax 定义为：

   $$
   \text{Softmax}(o_i) = \frac{e^{o_i}}{\sum_{k=1}^K e^{o_k}}.
   $$

2. **与交叉熵的结合**
   在多分类任务中，模型先计算每个类别的“得分”（logits）\\(o_i\\)，然后通过 Softmax 将这些得分转化为概率 \\(\hat{y}_i\\)。这样就可以把 \\(\hat{y}_i\\) 代入到前面所说的交叉熵损失里。
   - 目标是让真实类别对应的 \\(\hat{y}_j\\) 尽量接近 1
   - 其他类别的 \\(\hat{y}_i\\) 尽量接近 0

---

## 4. 损失函数的梯度推导

图片中提到的“其梯度是真实概率和预测概率的区别”指的是，当损失函数是 **交叉熵**，并且输出层使用 **Softmax** 时，对模型输出（logits）\\(\mathbf{o}\\) 求导会得到一个非常简洁的结果：

$$
\frac{\partial}{\partial o_i} \Bigl( -\sum_{k=1}^K y_k \log(\hat{y}_k) \Bigr) = \hat{y}_i - y_i.
$$

- \\(\hat{y}_i = \text{Softmax}(o_i)\\)
- \\(y_i\\) 是真实标签在第 \\(i\\) 类上的概率（one-hot 编码中，要么是 0，要么是 1）

**推导思路**：
1. 先求 \\(\hat{y}_i\\) 对 \\(o_j\\) 的偏导数：
   $$
   \frac{\partial \hat{y}_i}{\partial o_j} = \frac{\partial}{\partial o_j} \left(\frac{e^{o_i}}{\sum_{k} e^{o_k}}\right).
   $$
   当 \\(i = j\\) 与 \\(i \neq j\\) 的情况不同，最终可得矩阵形式的雅可比矩阵。
2. 将其代入到交叉熵损失的偏导中，最后得到 \\(\hat{y}_i - y_i\\)。

因为有了这样简洁的梯度，在 **反向传播** 时，参数更新更直接有效，这也是 **Softmax + 交叉熵** 这一组合在多分类问题中非常常用的原因之一。

---

## 5. 总结

- **Softmax**：将模型的线性输出（logits）映射为概率分布，使得输出的每个分量都介于 0 和 1 之间，并且总和为 1。
- **交叉熵损失**：衡量真实分布（one-hot 标签）与预测分布之间的差异，是分类问题中最常见、最有效的损失函数之一。
- **梯度**：在使用 Softmax 作为输出层、交叉熵作为损失函数的情况下，反向传播的梯度形式为 \\(\hat{y}_i - y_i\\)，有利于稳定且快速地训练模型。

在这张图片中，你能看到：

1. 交叉熵的公式 \\(\sum -p_i \log(q_i)\\)
2. 交叉熵损失（将真实标签视为 one-hot 分布）\\(-\sum y_i \log \hat{y}_i\\) 的常见简写 \\(-\log \hat{y}_y\\)
3. 梯度结果 \\(\text{softmax}(o) - y\\)，说明了模型在反向传播时更新参数的方向是「预测概率减去真实概率」。

这也解释了为什么「**Softmax + 交叉熵**」是一种高效的多分类模型输出与损失函数组合。
